In [2]:
# -*- coding: utf-8 -*-
# import sys
# # sys.setdefaultencoding() does not exist, here!
# reload(sys)  # Reload does the trick!
# sys.setdefaultencoding('UTF8')

In [73]:

import xgboost as xgb
from xgboost import XGBClassifier
from time import time

from time import time
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.svm import LinearSVC
from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor, LogisticRegression, PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

from sklearn.neural_network import MLPClassifier

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

from sklearn.model_selection import cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

In [4]:
def review_to_wordlist( review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, "lxml").get_text()

    #
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (True by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer #PorterStemmer()
    for word in words:
        b.append(stemmer.stem(word))

    # 5. Return a list of words
    return(b)

In [5]:
df = pd.read_excel('sentiment_train.xlsx')
print (df.shape)

holdout = pd.read_excel('sentiment_test.xlsx')
print (holdout.shape)

holdout['content'] = holdout['content'].apply(str)
df['content'] = df['content'].apply(str)

(25191, 2)
(8309, 1)


In [6]:
start=time()
clean_train_reviews = []
for review in df['content']:
    clean_train_reviews.append( " ".join(review_to_wordlist(review)))
    
clean_test_reviews = []
for review in holdout['content']:
    clean_test_reviews.append( " ".join(review_to_wordlist(review)))
    
print ('used: {:.2f}s'.format(time()-start))


used: 11.59s


In [9]:
clean_train_reviews[:10]

['funer ceremoni gloomi friday',
 'dannycastillo we want to trade with someon who has houston ticket but no one will',
 're ping ghostridah whi didn t you go to prom bc my bf didn t like my friend',
 'i should be sleep but im not think about an old friend who i want but he s marri now damn amp he want me scandal',
 'hmmm http www djhero com is down',
 'charviray charlen my love i miss you',
 'kelcouch i m sorri at least it s friday',
 'cant fall asleep',
 'brodyjenn if u watch the hill in london u will realis what tourtur it is becaus were week and week late i just watch itonlinelol',
 'got the news']

In [13]:
start=time()

# vectorizer = TfidfVectorizer(
# #     min_df=1, max_df=0.95,
#                              max_features = 1000000, ngram_range = ( 1, 5 ),
# #                               sublinear_tf = True 
# )

vectorizer = CountVectorizer( 
#     min_df=2, max_df=0.95, 
#                              max_features = 400000, 
                             ngram_range = ( 1, 5 ),
                              )

# vectorizer = HashingVectorizer( ngram_range = ( 1, 5 ),)

vectorizer = vectorizer.fit(clean_train_reviews+clean_test_reviews)
train_features = vectorizer.transform(clean_train_reviews)

test_features = vectorizer.transform(clean_test_reviews)

print ('used: {:.2f}s'.format(time()-start))
train_features

used: 6.99s


<25191x1085757 sparse matrix of type '<class 'numpy.int64'>'
	with 1292973 stored elements in Compressed Sparse Row format>

In [16]:
ori = train_features.copy()

In [36]:
fselect = SelectKBest(chi2 , k=65400)
train_features = fselect.fit_transform(ori, df["sentiment"])

test_features = fselect.transform(test_features)

In [37]:
start = time()
model5 = LogisticRegression(
#     solver='lbfgs'
)
# model5.fit( train_features, train["sentiment"] )
cv_result = cross_val_score( model5, train_features, df['sentiment'], cv=5)

print ('used: {:.2f}s'.format(time()-start))
cv_result

used: 7.46s


array([ 0.41150794,  0.44770788,  0.49364827,  0.4837237 ,  0.43963463])

In [78]:
start = time()
model5 = XGBClassifier(
    silent=0
#     solver='lbfgs'
)
# model5.fit( train_features, train["sentiment"] )
cv_result = cross_val_score( model5, train_features, df['sentiment'], cv=5)

print ('used: {:.2f}s'.format(time()-start))
cv_result

used: 33.43s


array([ 0.3640873 ,  0.3826156 ,  0.4162366 ,  0.41127432,  0.37787927])

In [39]:
start = time()
model5 = MLPClassifier(
    early_stopping=True,
    verbose=1
#     solver='lbfgs'
)
model5.fit( train_features, df["sentiment"] )
# cv_result = cross_val_score( model5, train_features, df['sentiment'], cv=5)

print ('used: {:.2f}s'.format(time()-start))
# cv_result

Iteration 1, loss = 1.55794288
Validation score: 0.446825
Iteration 2, loss = 1.13033878
Validation score: 0.472619
Iteration 3, loss = 0.91946503
Validation score: 0.482937
Iteration 4, loss = 0.80483726
Validation score: 0.501190
Iteration 5, loss = 0.72891243
Validation score: 0.492063
Iteration 6, loss = 0.67245485
Validation score: 0.494444
Iteration 7, loss = 0.62897122
Validation score: 0.500794
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
used: 79.31s


In [59]:
label = pd.factorize(df['sentiment'])[0]
pd.factorize(df['sentiment'])

(array([0, 1, 2, ..., 1, 5, 5]),
 Index(['sadness', 'neutral', 'worry', 'surprise', 'happiness', 'love'], dtype='object'))

In [60]:
X_train, X_valid, y_train, y_valid = train_test_split(train_features, label, 
                                                      test_size = 0.20, 
                                                      random_state = 345)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((20152, 65400), (5039, 65400), (20152,), (5039,))

In [98]:
start = time()
model23 = MLPClassifier(
    early_stopping=True,
    verbose=1
#     solver='lbfgs'
)
model23.fit( X_train, y_train )
# cv_result = cross_val_score( model5, train_features, df['sentiment'], cv=5)

print ('used: {:.2f}s'.format(time()-start))
# cv_result

Iteration 1, loss = 1.60635918
Validation score: 0.444940
Iteration 2, loss = 1.17966750
Validation score: 0.478671
Iteration 3, loss = 0.94606176
Validation score: 0.499008
Iteration 4, loss = 0.81789545
Validation score: 0.504960
Iteration 5, loss = 0.72829096
Validation score: 0.496528
Iteration 6, loss = 0.66206185
Validation score: 0.502480
Iteration 7, loss = 0.61222124
Validation score: 0.505456
Iteration 8, loss = 0.57023453
Validation score: 0.495536
Iteration 9, loss = 0.53747579
Validation score: 0.501488
Iteration 10, loss = 0.50950245
Validation score: 0.501488
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
used: 90.43s


In [101]:
model23.score(X_train, y_train)

0.78339618896387453

In [105]:
model23.score(X_valid, y_valid)

0.48422306013097838

In [83]:
d_train = xgb.DMatrix(x_train, y_train)
d_valid = xgb.DMatrix(x_valid, y_valid)

d_test = xgb.DMatrix(test_features)

In [86]:
d_train.get_label()

array([ 0.,  1.,  5., ...,  4.,  4.,  2.], dtype=float32)

In [87]:
accuracy_score(d_train.get_label(), y_train)

1.0

In [106]:
RANDOM_STATE = 2017
params = {
#     'min_child_weight': 1,
#     'eta': 0.01,
#     'colsample_bytree': 0.5,
#     'max_depth': 13,
#     'subsample': 0.8,
#     'alpha': 1,
#     'gamma': 1,
#     'silent': 0,
    'learning_rate':0.5,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"multi:softmax", 
    'num_class':6
    
}
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

def xg_eval(pred, dtrain):
    y = dtrain.get_label()
    return 'acc', accuracy_score(y, pred)

In [107]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=150, 
                verbose_eval=20, feval=xg_eval, maximize=True)
                


'!!! used %.2f s'%(time()-start)

[0]	train-acc:0.388001	valid-acc:0.380036
Multiple eval metrics have been passed: 'valid-acc' will be used for early stopping.

Will train until valid-acc hasn't improved in 150 rounds.
[20]	train-acc:0.505161	valid-acc:0.4102
[40]	train-acc:0.557116	valid-acc:0.413376
[60]	train-acc:0.59076	valid-acc:0.412979
[80]	train-acc:0.616068	valid-acc:0.416749
[100]	train-acc:0.635768	valid-acc:0.418535
[120]	train-acc:0.65264	valid-acc:0.419726
[140]	train-acc:0.666534	valid-acc:0.418932
[160]	train-acc:0.679933	valid-acc:0.421711
[180]	train-acc:0.691693	valid-acc:0.423497
[200]	train-acc:0.701717	valid-acc:0.422306
[220]	train-acc:0.7105	valid-acc:0.420718
[240]	train-acc:0.71839	valid-acc:0.420123
[260]	train-acc:0.727124	valid-acc:0.421512
[280]	train-acc:0.733873	valid-acc:0.419329
[300]	train-acc:0.742408	valid-acc:0.418734
[320]	train-acc:0.748313	valid-acc:0.419925
Stopping. Best iteration:
[184]	train-acc:0.693827	valid-acc:0.424687



'!!! used 42.83 s'

In [99]:
clf.predict(d_valid)

array([ 1.,  4.,  1., ...,  0.,  5.,  1.], dtype=float32)